#### import functions

In [1]:
%run functions_NoiseStudy.py
%matplotlib 

# global variables for plot savings
fs = 13
type_plot = ['png', 'svg']

Using matplotlib backend: MacOSX


### 0. Directory for data storage

In [2]:
# actual evaluation time
now = datetime.datetime.now()
today = now.strftime("%Y%m%d")

# output folder
save_dir_plots = 'plots/' + today + '_measurement'
if not os.path.exists(save_dir_plots):
    os.makedirs(save_dir_plots)
    
save_dir_res = 'Results/' + today + '_measurement'
if not os.path.exists(save_dir_res):
    os.makedirs(save_dir_res)

### 1. Load data

In [3]:
# [USER INPUT]
# calibration data 
file_calib = 'Results/20210531_calibration/20210531-1455_calibration_1RoI.hdf5'

# measurement data - images of the optode
file_meas = '/Volumes/HIS-CAMEL/04measurementData/20201127_Noise-vs-resolution-paper/Klaus_Optode_noise_study_26-11-2020/O2gradient_experiement/'


##### additional information

In [4]:
# image cropping to region of interest (RoI) that should be analyzed
RoI_op = [[(730, 200), (730, 1250), (1290, 1250), (1290, 200)],   # optode1
          [(1560, 200), (1560, 1250), (2100, 1250), (2100, 200)]] # optode2

# Image resolution - determine conversion factor px -> mm - user definition using
# f.ex. imageJ
px2mm, dpi = image_resolution(px=840.6646, dist_mm=30., inch=1.1811)
print('Image resolution: 1cm equals: {:.2f}px, i.e. {:.0f}dpi'.format(px2mm, dpi))

Image resolution: 1cm equals: 28.02px, i.e. 712dpi


In [27]:
# load calibration data
# dic_header, dinitial, dnorm, dpara = load_calibResults(path_calib=file_calib)
calib_info = load_calibration_para_v1(path_calib=file_calib)

# load measurement data and crop according to RoI
dint_red, dint_green, dint_ratio = splitImage(path=file_meas, RoI_op=RoI_op)

### 2. Determine O2 concentration in each pixel and each measurement
using the Stern-Volmer equation and the respective calibration fit

In [6]:
# one example of one optode
inp = input('Which optode and which setting shall be visualized? ') # optode1, set2
run = input('Which measurement shall be visualized - 1 or 2?') # 1,2

Which optode and which setting shall be visualized? optode1, set2
Which measurement shall be visualized - 1 or 2?1


In [46]:
# [USER input] - define surface depth within the image
surface = (12.9, 12.) # optode 1, optode2
depth_op =  (-1.5, 1.)  # depth for optode excerpt

# .................
# Oxygen concentration determination
dO2_optode = O2blur_optode(kshape=(1,1), inp=inp, path_calib=file_calib, px2mm=px2mm, kernel='gauss', 
                           dint_ch1=dint_ratio[inp.split(',')[0]][inp.split(',')[-1].strip()], dint_ch2=None,
                           surface=surface, blur_pos='ratio', depth_min=depth_op[0], depth_max=depth_op[1])
# as no smoothing is applied, "vertical" eqauls "horizontal" equals "square" for kernel size (1,1)
dO2_av = dO2_optode['vertical']

In [60]:
dO2_av

,0.000000,0.035686,0.071372,0.107058,0.142744,0.178430,0.214116,0.249802,0.285488,0.321174,...,19.663014,19.698700,19.734386,19.770072,19.805758,19.841444,19.877131,19.912817,19.948503,19.984189
-1.480464,94.707012,102.040317,78.641021,109.760446,73.535756,79.899726,90.118453,86.395306,85.923763,78.957696,...,89.562774,69.608157,77.742761,77.058256,86.595742,92.276039,88.811197,75.827904,88.973849,91.636316
-1.444778,80.289263,90.770068,99.858939,106.019282,89.006502,79.173702,92.136085,94.627441,95.236847,89.752932,...,76.254703,71.576086,83.719706,78.946650,97.014046,94.652597,101.349149,79.225407,97.679229,82.518598
-1.409091,91.024389,83.483206,92.335533,84.790061,88.862725,82.480440,90.000043,95.397937,81.868759,76.699333,...,94.505311,93.600390,80.634304,91.419083,94.157148,89.550552,94.003060,78.853040,89.988954,76.798021
-1.373405,79.445171,88.644544,107.550402,81.519090,94.662008,96.737270,82.310607,82.788777,86.469666,85.798378,...,85.105973,83.996412,101.425490,73.620249,79.537140,86.974718,94.513496,97.765754,76.385487,76.820566
-1.337719,81.853342,81.959790,84.059679,84.133692,83.642581,73.322107,102.322479,83.425353,86.709541,87.995074,...,72.853787,90.100340,88.422692,75.006813,84.101891,86.857555,80.680914,89.747453,80.770347,72.487104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.839130,1.336895,-2.230803,-2.605292,2.248179,-1.749196,-2.137609,-2.447654,-0.132375,-1.587720,-0.450701,...,-1.217033,-0.666225,-0.455462,2.116820,1.742889,-0.172223,-1.364208,-0.505624,1.500612,-1.157526
0.874816,0.571206,-1.784507,-1.516088,-1.498679,-0.079172,-2.419714,0.374254,-3.282860,0.179761,-2.931869,...,1.042887,-0.641173,-0.320945,0.499115,1.386234,-2.090485,-1.267280,-1.747310,0.640243,-0.079711
0.910502,3.753327,-0.132641,2.644410,-2.230803,-0.595828,-0.853254,-3.347875,-1.382407,1.893864,4.114935,...,1.437801,1.119078,-1.944211,0.268122,-1.247613,3.216222,2.326098,2.842849,-1.031964,-1.748455
0.946188,-3.218953,-0.859625,-2.931869,1.151186,-1.200780,2.327849,0.139257,1.385405,-1.419156,-1.260727,...,1.216883,4.239003,-0.505624,0.219790,1.471347,-0.127811,-1.041700,1.641549,1.390044,0.383708


### 3. Plot optode as 3D or 2D

In [43]:
# 3D visualization
sns.set_style('darkgrid')
fig3D, ax3D = plot_wholeImage3D(dO2_mean=dO2_av, unit='mm', pad=2)
plt.show()   

In [74]:
# 2D projection
depth_range = (-12.5, -10.5)
width_range = (2., 20.)

sns.set_style('ticks')
fig2D, ax2D = plot_optode2D(o=inp.split(',')[0], s=inp.split(',')[1].strip(), px2mm=px2mm, 
                            surface=surface, dO2_av=dO2_av, depth_range=depth_range,
                            width_range=width_range, vmin=0, vmax=100)
plt.show() 

### 4. Save results
save figures

In [51]:
now = datetime.datetime.now() 
name3D = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_optode3D_' + '-'.join([i.strip() for i in inp.split(',')])
name2D = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_optode2D_' + '-'.join([i.strip() for i in inp.split(',')])
for i in type_plot:
    fig3D.savefig(name3D + '.{}'.format(i), dpi=dpi) 
    fig2D.savefig(name2D + '.{}'.format(i), dpi=dpi) 

save determined O2

In [52]:
eval_time = now.strftime("%Y%m%d-%H%M")
save_name = save_dir_res + '/' + eval_time + '_O2calc_run' + run + '.hdf5'
save_o2results(save_name=save_name, inp=inp, file_meas=file_meas, RoI_op=RoI_op,
               px2mm=px2mm, surface=surface, dO2_av=dO2_av, dO2_SD=dO2_SD)

saving done
